In [1]:
# from randtest import randtest
import numpy as np
import pandas
import pickle
from tqdm import tqdm
import random
import tensorflow.keras.backend as K

2024-03-28 17:40:00.808561: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 17:40:00.857182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 17:40:02.108065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
print(cores)

80


In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
def forecast_loss(y_true, y_pred):
    V=134
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

In [5]:
def score(y_preds, y_true):
    y_pred = y_preds.reshape(-1, 134)
    mmse = forecast_loss(y_true, y_pred)
    s = 0
    for i in mmse:
        s += i
    return s/len(mmse)

In [6]:
def ra_test(a_0, b_0, y_true, r=1000):
    """randomization test"""
    if len(a_0) != len(b_0):
        print('Input A and B are not eaqul-sized!')
        return
    delta_s_rs = []
    c = 0
    n = len(a_0)
    
    
    delta_s_0 = score(a_0, y_true) - score(b_0, y_true)
    delta_s_0 = delta_s_0.numpy()
    
    for _ in tqdm(range(r)):
        a_r = []
        b_r = []
        # shuffle
        for i in range(n):
            # swap
            rn = random.randint(0, 1)
            # swap
            if rn == 0:
                a_r.append(b_0[i])
                b_r.append(a_0[i])
            # do not swap
            else:
                a_r.append(a_0[i])
                b_r.append(b_0[i])

        a_r = np.array(a_r)
        b_r = np.array(b_r)
        delta_s_r = score(a_r, y_true) - score(b_r, y_true)
        delta_s_r = delta_s_r.numpy()
        # print(delta_s_r)
        # print(abs(delta_s_r))
        delta_s_rs.append(delta_s_r)
        if abs(delta_s_r) >= abs(delta_s_0):
            c += 1
        
    print(c)
    p = c/r
    print(delta_s_rs)
    return p

In [7]:
baseline_path = 'randomization_test/data/baseline.pkl'
baseline_data, var_to_ind = pickle.load(open(baseline_path, 'rb'))

forecasting_preds = []

# y_true for score
y_true = []
for y in baseline_data['forecasting_test_op']:
    y_true.append(y)
y_true = np.array(y_true)

for pred in baseline_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)
x = forecasting_preds.flatten()
del baseline_data, forecasting_preds

In [8]:
m_path = 'randomization_test/data/post_koll_exp_1_concat.pkl'
m_data, var_to_ind_m = pickle.load(open(m_path, 'rb'))
m_data

forecasting_preds = []

for pred in m_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)

y = forecasting_preds.flatten()
del m_data, forecasting_preds

In [9]:
p = ra_test(x, y, y_true = y_true, r=100)
p

2024-03-28 17:40:21.791936: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31133 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2024-03-28 17:40:21.792549: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31133 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b3:00.0, compute capability: 7.0
100%|██████████| 100/100 [2:12:01<00:00, 79.21s/it] 

6
[-0.0037736221998070363, 0.011971172570126143, 0.019477066407849186, -0.0009982409435513873, 0.00854048137908503, -0.014566150598407823, -0.0070087153169478, 0.021640487768315175, -0.00605486373755415, 5.181206164373009e-05, 0.008703283118593141, -0.013514854488579253, -0.01002066104618482, 0.01485334238732694, 0.007361083450849293, -0.01660028475516473, 0.00416843724094651, 0.007273916700906824, 0.003051642525178977, 0.011584697327641003, 0.0035954165361404833, 0.00585384915163889, -0.0030549084156703543, 0.00017799269332030576, 0.003499833071546554, -0.009011254268732394, 0.007794708662363092, 0.001758005476308, -0.011218876023716184, 0.0127049762073419, -0.0077947116094563285, 0.00884325591481705, -0.008586504387380067, 0.007563656766691018, 0.011850664223140406, -0.008377322408751553, 0.002674841059910449, 0.014269794045158157, 0.01342713385715033, -0.007233579091922238, -0.0005797568188077307, 0.004306069685569014, 0.004601863877770285, -0.008018839256263455, -0.0144315102864123

0.06

In [10]:
p

0.06

In [11]:
p

0.06